In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn import preprocessing

In [80]:
iris_data = datasets.load_iris()

X = iris_data.data
y = iris_data.target.astype("float")
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.33,
                                                    random_state=42)

# scaler = preprocessing.StandardScaler().fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

In [81]:
class DecisionNode:
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
    
class LeafNode:
    def __init__(self, X, y):
        classes, counts = np.unique(y, return_counts=True)
        self.predictions = dict(zip(classes, counts))    

class SplitQuestion:
    def __init__(self, column, value):
        self.column = column
        self.value = value
    
    def match(self, X):
        # Compare the feature value in an example to the
        # feature value in this question.
        matches = X[:, self.column]
        return matches >= self.value
    
    def match_row(self, example):
        val = example[self.column]
        return val >= self.value
    

In [82]:
class DecisionTreeClassifier:
    def __init__(self):
        self.is_built = False
    
    @staticmethod
    def gini(X, y):
        n = float(len(X))
        classes, counts = np.unique(y_train, return_counts=True)
        class_counts = dict(zip(classes, counts))
        impurity = 1
        for cl, cnt in class_counts.items():
            prob_of_lbl = cnt / n
            impurity -= prob_of_lbl**2
        
        return impurity
    
    def partition(self, X, y, question):
        matches = question.match(X)
        
        true_X, true_y = X[matches], y[matches]
        false_X, false_y = X[~matches], y[~matches]

        return true_X, true_y, false_X, false_y
    
    def info_gain(self, true_X, true_y, false_X, false_y, current_uncertainty):
        num_true = len(true_X)
        num_false = len(false_X)
        p = float(num_true) / (num_true + num_false)
        return current_uncertainty - p * self.gini(true_X, true_y) - (1 - p) * self.gini(false_X, false_y)
        
    def find_best_split(self, X, y):
        best_gain = 0
        best_question = None
        current_uncertainty = self.gini(X, y)
        n_features = X.shape[1]
        ret_true_X, ret_true_y, ret_false_X, ret_false_y = None, None, None, None

        for col in range(n_features):
            unique_vals = np.unique(X[:, col])
            for val in unique_vals:
                question = SplitQuestion(col, val)

                # try splitting the dataset
                true_X, true_y, false_X, false_y = self.partition(X, y, question)

                # skip if data are not split
                if len(true_X) == 0 or len(false_X) == 0:
                    continue

                gain = self.info_gain(true_X, true_y, false_X, false_y, current_uncertainty)

                # Can be either '>' or '>='
                if gain >= best_gain:
                    best_gain, best_question = gain, question
                    ret_true_X, ret_true_y, ret_false_X, ret_false_y = true_X, true_y, false_X, false_y

        return best_gain, best_question, ret_true_X, ret_true_y, ret_false_X, ret_false_y
    
    def build_tree(self, X, y):
        gain, question, true_X, true_y, false_X, false_y = self.find_best_split(X, y)
        
        if gain == 0:
            return LeafNode(X, y)
        
        # Recursively build the true branch.
        true_branch = self.build_tree(true_X, true_y)

        # Recursively build the false branch.
        false_branch = self.build_tree(false_X, false_y)

        # Return a DecisionNode node.
        # This records the best feature / value to ask at this point,
        # as well as the branches to follow
        # depending on the answer.
        return DecisionNode(question, true_branch, false_branch)
    
    def fit(self, X, y):
        self.model = self.build_tree(X, y)
        self.is_built = True
        
    def predict(self, X):
        y_pred = []
        for x in X:
            y_p = self.classify(x, self.model)
            y_p = list(y_p.values())
            y_pred.append(y_p)
            
        return np.concatenate(y_pred, axis=0)
        
    def classify(self, x, node):
        if isinstance(node, LeafNode):
            return node.predictions

        # Decide whether to follow the true-branch or the false-branch.
        # Compare the feature / value stored in the node,
        # to the example we're considering.
        if node.question.match_row(x):
            return self.classify(x, node.true_branch)
        else:
            return self.classify(x, node.false_branch)
    

In [83]:
tree_clf = DecisionTreeClassifier()
tree_clf.fit(X_train, y_train)

In [84]:
y_pred = tree_clf.predict(X_test)
f1_score(y_test, y_pred, average="micro")

0.3